## Creation of the mangroves extent distance asset

In [1]:
# @deprecated : there is a new method for calculating the coastal length data
import os
from pathlib import Path
import json
import ee
from typing import List
import geemap
import geopandas as gpd
import shapely.speedups
shapely.speedups.enable()
%run utils.ipynb

In [ ]:
# Trigger the authentication flow.
#ee.ServiceAccountCredentials(EE_ACCOUNT, EE_PRIVATE_KEY_FILE)
ee.Authenticate()
# Initialize the library.
ee.Initialize()

In [2]:
# LOCAL PATHS
#  FIXME: This will depends from where the notebook kernel is running so be careful
WORK_DIR =Path(os.getcwd())
BASE_DIR = f'{WORK_DIR.parents[3]}/datasets/raw'

# @TODO: Add expected data files source as an environment variable.
assert BASE_DIR == '/home/jovyan/work/datasets/raw', f'{BASE_DIR} is not the correct directory'

# variables
dataset = 'mangrove_blue_carbon'

# GOOGLE CLOUD STORAGE PATHS


# GOOGLE EARTH ENGINE ASSET PATHS


# SET EXPORT REGION
region = ee.Geometry.Polygon([[[-180, 33],[-180, -34],[180, -34],[180, 33]]], None, False)

EEException: Earth Engine client library not initialized. Run `ee.Initialize()`

In [ ]:
  # CALCULATIONS, derived from mangrove extent dataset
def calculateLinearDistance(im: ee.Image, maxDistance: int) -> ee.Image:

    dist_pixels = im.fastDistanceTransform(
                        maxDistance, 'pixels', 'squared_euclidean'
                    ).sqrt()
    pixel_size_m = im.pixelArea().sqrt()
    
    return dist_pixels.multiply(pixel_size_m)

def calc_dist(imageCollection: ee.ImageCollection, year: int, image_properties: ImageProperties, run_task: bool):
    # time_start
    ts = ee.Date.fromYMD(year, 1, 1)
    # get image
    image = ee.Image(imageCollection.filterDate(ts).first()).selfMask()
    # calculate distance
    dist = calculateLinearDistance(image, 256)
    
    # mask to 65km and convert to uint16
    dist = dist.rename(image_properties.band_name
    ).updateMask(dist.lte(65000)
    ).uint16(
    ).set({
      'system:time_start': ts
        }
    ).set(image_properties)

    # Export params
    year_string = ee.Number(year).format()
    nm = ee.String("gl_").cat(year_string).cat("_").cat('distance').getInfo()
    ns = image.projection().nominalScale().getInfo()
    
    # EXPORT TO IMAGE COLLECTION
    params = {
        'image': dist,
        'description': f"export_{nm}",
        'assetId': f'projects/global-mangrove-watch/land-cover/mangrove-extent-distance_version-2-0_1996--2016/{nm}',
        'pyramidingPolicy':{image_properties.band_names: image_properties.band_pyramiding_policies},
        'scale': ns,
        'crs': 'EPSG:4326',
        'region': region,
        'maxPixels': 1e13
      
    }
    task = ee.batch.Export.image.toAsset(**params)
    
    if debug == False:
        task.start()
        print(task.status())
    
    return 'true'

def createLinearDistanceAsset(extent: ee.ImageCollection, image_collection_properties: ImageCollectionProperties, run_task: bool):
    """
    Calculate the linear distance of a geometry.
    """
    # Loop through year list
    for year in year_list:
        calc_dist(year, run_task)
    return geometry.length